In [2]:
from graphviz import Digraph

#### Comparison of RF and NN ensemble

In [6]:
dag = Digraph(node_attr={'shape': 'rectangle', 'fontname': 'courier'})

fig_kwargs = {
    "shape": "oval", 
    "color": "blue",
    "fontcolor": "blue",
    "fontname": "helvetica"
}

# workflow steps
dag.node(name="nudging-run", label="make nudge_to_fine_rad_precip_prescribed")
dag.node(name="train-data-zarrs", label="make nudge_to_fine_training_data_zarrs")
dag.node(name="train-rf", label="make train_rf")
dag.node(name="train-nn-seeds", label="make train_nn_random_seeds")
dag.node(name="prog-nn-ensemble", label="make prognostic_nn_ensemble_ics")
dag.node(name="nn-ensemble-offline-report", label="make offline_report_nn_ensemble")

dag.node(name="prog-rf", label="make prognostic_rf_ics")
dag.node(name="prog-report", label="make prognostic_report_rf_nn_comparison")

dag.edges(
    [
        ("nudging-run", "train-data-zarrs"),
        ("train-data-zarrs", "train-rf"),
        ("train-data-zarrs", "train-nn-seeds"),
        ("train-nn-seeds", "nn-ensemble-offline-report"),
        ("train-rf", "prog-rf"),
        ("train-nn-seeds", "prog-nn-ensemble"),
        ("prog-rf", "prog-report"),
        ("prog-nn-ensemble", "prog-report"),
        
    ]
)

dag.render("../main-experiment-dag", view=True, format="png", cleanup=True)

'../main-experiment-dag.png'

#### Sensitivity to prescribed surface fluxes and precip in nudging run, 

In [7]:
dag = Digraph(node_attr={'shape': 'rectangle', 'fontname': 'courier'})

done_kwargs = {
    "color": "grey",
    "fontcolor": "grey",
}

# workflow steps
dag.node(name="nudging-run-prescribed", label="make nudge_to_fine_rad_precip_prescribed", **done_kwargs)
dag.node(name="nudging-run-control", label="make nudge_to_fine_rad_precip_control")

dag.node(name="train-data-zarrs-prescribed", label="make nudge_to_fine_training_data_zarrs_prescribed", **done_kwargs)
dag.node(name="train-data-zarrs-control", label="make nudge_to_fine_training_data_zarrs_control")

dag.node(name="train-prescribed", label="make train_rf", **done_kwargs)
dag.node(name="train-control", label="make train_rf_control")

dag.node(name="prog-prescribed-ml-rad", label="make prognostic_rf_ics", **done_kwargs)

dag.node(name="prog-control-ml-rad", label="make prognostic_training_control_ml_tendencies_rad")
dag.node(name="prog-control-ml-tendencies-only", label="make prognostic_training_control_ml_tendencies_only")

dag.node(name="prog-report", label="make prognostic_report_sensitivity")

dag.edge("nudging-run-prescribed", "train-data-zarrs-prescribed", **done_kwargs)
dag.edge("train-data-zarrs-prescribed", "train-prescribed",  **done_kwargs)
dag.edge("train-prescribed", "prog-prescribed-ml-rad", **done_kwargs)
dag.edges(
    [
        ("nudging-run-control", "train-data-zarrs-control"),
        
        ("train-data-zarrs-control", "train-control"),
        
        ("train-control", "prog-control-ml-rad"),
        ("train-control", "prog-control-ml-tendencies-only")
        
    ]
)
dag.edges([
    ("prog-control-ml-rad", "prog-report"),
    ("prog-prescribed-ml-rad", "prog-report"),
    ("prog-control-ml-tendencies-only", "prog-report")
])
dag.render("../sensitivity-experiment-dag", view=True, format="png", cleanup=True)

'../sensitivity-experiment-dag.png'